In [ ]:
# pip install Wikipedia-API==0.5.4

In [54]:
import wikipediaapi
from tqdm import tqdm
import pandas as pd
import numpy as np
import re

[nltk_data] Downloading package stopwords to /home/dojo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Download from kaggle.com/tunguz/200000-jeopardy-questions
jp = pd.read_csv('data/raw_data/JEOPARDY_CSV.csv')

In [3]:
jp.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [4]:
pattern = re.compile(r'\ban\b|\bthe\b|\ba\b')

In [5]:
jp['wiki'] = jp[' Answer'].apply(lambda x: pattern.sub('', str(x)))
jp

,Show Number,Air Date,Round,Category,Value,Question,Answer,wiki
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,John Adams
...,...,...,...,...,...,...,...,...
216925,4999,2006-05-11,Double Jeopardy!,RIDDLE ME THIS,$2000,This Puccini opera turns on the solution to 3 ...,Turandot,Turandot
216926,4999,2006-05-11,Double Jeopardy!,"""T"" BIRDS",$2000,In North America this term is properly applied...,a titmouse,titmouse
216927,4999,2006-05-11,Double Jeopardy!,AUTHORS IN THEIR YOUTH,$2000,"In Penny Lane, where this ""Hellraiser"" grew up...",Clive Barker,Clive Barker
216928,4999,2006-05-11,Double Jeopardy!,QUOTATIONS,$2000,"From Ft. Sill, Okla. he made the plea, Arizona...",Geronimo,Geronimo


In [6]:
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [7]:
def get_wiki_contents(answer, clean_answer):
    article = wiki_wiki.page(answer)
    if len(article.text) == 0:
        article = wiki_wiki.page(clean_answer)
    return article.text

In [9]:
for i in tqdm(range(len(jp))):
    jp.iloc[i, jp.columns.get_loc('wiki')] = get_wiki_contents(jp[' Answer'][i], jp['wiki'][i])

100%|██████████| 100/100 [02:34<00:00,  1.55s/it]


In [ ]:
df = jp.copy()

Remove null articles

In [73]:
df['wiki'] = df['wiki'].replace('', np.nan)

In [74]:
df = df.dropna().copy().reset_index(drop=True)

In [75]:
df

,Show Number,Air Date,Round,Category,Value,Question,Answer,wiki
0,5482,2008-06-10,Double Jeopardy!,THE MIDDLE EAST,$800,In 1971 this federation of 6 states gained its...,the United Arab Emirates,The United Arab Emirates (UAE; Arabic: الإمارا...
1,5482,2008-06-10,Double Jeopardy!,JAPANESE CUISINE,$800,Cows for this type of beef drink beer to stimu...,Kobe,Kobe ( KOH-bay; Japanese: [koꜜːbe]; officially...
2,5482,2008-06-10,Double Jeopardy!,BIOLOGY,$800,"Measles, mumps & rabies are not transmitted by...",viruses,A virus is a submicroscopic infectious agent t...
3,5482,2008-06-10,Double Jeopardy!,TRACK & FIELD,$1200,"In this event, though athletes regularly clear...",the pole vault,"Pole vaulting, also known as pole jumping, is ..."
4,5482,2008-06-10,Double Jeopardy!,THE MIDDLE EAST,$1200,He assumed the presidency of Syria upon the de...,Bashar Assad,Bashar Hafez al-Assad (born 11 September 1965)...
...,...,...,...,...,...,...,...,...
79,3321,1999-02-01,Jeopardy!,THE QUEEN'S ENGLISH,$200,"A dustman doesn't put you to sleep, he collect...",Garbage,"Garbage, trash, rubbish, or refuse is waste ma..."
80,3321,1999-02-01,Jeopardy!,BLACK HISTORY,$300,In 1829 journalist John Russwurm gave up on ab...,Liberia,"Liberia ( (listen)), officially the Republic o..."
81,3321,1999-02-01,Jeopardy!,MAKING A LIST,$300,"A line from this 1993 movie is ""The list is life""",Schindler's List,Schindler's List is a 1993 American epic histo...
82,3321,1999-02-01,Jeopardy!,SCARY AUTHORS,$300,"""Memnoch The Devil"", ""The Queen Of The Damned""...",Anne Rice,Anne Rice (born Howard Allen Frances O'Brien; ...


In [76]:
df.to_pickle('data/combined-dataset.pkl')